# Try fine-tuning `Sentence BERT`
Since Thur. Dec. 9th, 2021

To set up our fine-tuning pipeline

To verify our implementation, check if we can reproduce the results from [*Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks*](https://arxiv.org/abs/1908.10084).
With Hugging Face dependency, not [sentence-transformers](https://github.com/UKPLab/sentence-transformers).

Specifically, on **section 4.2 Supervised STS**, go with `SBERT-STSb-base`:
> The STS benchmark (STSb) (Cer et al., 2017) provides is a popular dataset to evaluate supervised STS systems.

> We use the training set to fine-tune SBERT using the regression objective function. At prediction time, we compute the cosine-similarity between the sentence embeddings. All systems are trained with 10 random seeds to counter variances.

An example on [sentence-transformers](https://github.com/UKPLab/sentence-transformers):
    [training_stsbenchmark.py](https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/sts/training_stsbenchmark.py).


## Setup



In [6]:
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset
from icecream import ic

from util import *


name_tune = 'eg_sbert'
d_tune = config(f'fine-tune.{name_tune}')
dnm = d_tune["dataset_name"]
d_dset = config(f'datasets.{dnm}')
md_path = os.path.join(PATH_BASE, DIR_MDL, name_tune)
mdnm = d_tune['embedding_model_name']

seed = config('random-seed')



## Check STSb data



In [7]:
dset = load_dataset(dnm, name='en')
ic(dset)
ic(len(dset['train']), dset['train'][0])



Reusing dataset stsb_multi_mt (/Users/stefanh/.cache/huggingface/datasets/stsb_multi_mt/en/1.0.0/a5d260e4b7aa82d1ab7379523a005a366d9b124c76a5a5cf0c4c5365458b0ba9)


  0%|          | 0/3 [00:00<?, ?it/s]

ic| dset: {'dev': Dataset({
              features: ['sentence1', 'sentence2', 'similarity_score'],
              num_rows: 1500
          }),
           'test': Dataset({
              features: ['sentence1', 'sentence2', 'similarity_score'],
              num_rows: 1379
          }),
           'train': Dataset({
              features: ['sentence1', 'sentence2', 'similarity_score'],
              num_rows: 5749
          })}
ic| len(dset['train']): 5749
    dset['train'][0]: {'sentence1': 'A plane is taking off.',
                       'sentence2': 'An air plane is taking off.',
                       'similarity_score': 5.0}


(5749,
 {'sentence1': 'A plane is taking off.',
  'sentence2': 'An air plane is taking off.',
  'similarity_score': 5.0})

## Check BERT model


In [8]:
tokenizer = AutoTokenizer.from_pretrained(mdnm)
ic(tokenizer)

model = AutoModel.from_pretrained(mdnm)
assert str(model.__class__) == "<class 'transformers.models.bert.modeling_bert.BertModel'>"



ic| tokenizer: PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect t